In [ ]:
import torch 
from sentence_transformers import SentenceTransformer
import faiss
from faiss import write_index, read_index
import polars as pl

In [ ]:
# torch.device('cuda')

In [ ]:
model = SentenceTransformer("inkoziev/sbert_pq", device=torch.device('cuda'))

In [ ]:
import os
os.getcwd()

In [ ]:
videos_t = pl.read_parquet('../train_data/videos.parquet',columns=['video_title']).select(pl.col('video_title').str.to_lowercase())

In [ ]:
corpus = videos_t['video_title'].to_list()

In [ ]:
import math
batch_size = 200000
num_batches = math.ceil(len(corpus) / batch_size)

In [ ]:
num_batches

In [ ]:
import gc
gc.collect()

In [ ]:
import faiss
from faiss import write_index, read_index
faiss.omp_get_max_threads()

In [ ]:
d = 312
cpu_index = faiss.IndexFlatL2(d)


In [ ]:
# res = faiss.StandardGpuResources()  # use a single GPU
# gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, cpu_index)

In [ ]:
model.eval()

In [ ]:
with torch.no_grad():
    if True:
        try:
            for i in range(num_batches):
                # формируем батч
                start, end = i * batch_size, (i + 1) * batch_size
                corpus_batch = corpus[start:end]

                # считаем вектора для всех предложений в батче
                embeddings = model.encode(
                    corpus_batch,
                    batch_size=40000,
                    show_progress_bar=True
                )

                # добавляем новые батч векторов в индекс и сохраняем его
                cpu_index.add(embeddings)

                if i%10==0:
                    write_index(cpu_index, 'candidates.index')


                print(f'batch: {i + 1} / {num_batches}, vectors: {cpu_index.ntotal}')

                del embeddings
                gc.collect()
                torch.cuda.empty_cache()
        except KeyboardInterrupt:
            print('Остановлено пользователем')
            try:
                del embeddings
            except:
                pass

In [ ]:
# cpu_index = faiss.index_gpu_to_cpu(gpu_index_flat)  # error happens here
write_index(cpu_index, 'candidates.index')